In [1]:
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QMenuBar, QMenu, QAction, QWidget
from PyQt5.QtGui import QIcon,QPixmapwidth
import sys
import os
from PyQt5 import uic

class Window(QMainWindow):
    def __init__(self):
        super(Window, self).__init__()

        # Load the ui file
        uic.loadUi("image.ui", self)

        # Define our widgets
        self.label = self.findChild(QLabel, "label")


        openAction = QAction(QIcon('open.png'), '&Open', self)
        openAction.triggered.connect(self.openCall)
        
        self.b = self.findChild(QPushButton, "pushButton")
        menuBar = self.menuBar()
        fileMenu = menuBar.addMenu('&File')
        fileMenu.addAction(openAction)
        
        # Show The App
        self.show()
        
        
        

    def openCall(self):
        fname = QFileDialog.getOpenFileName(self, "Open File", os.getcwd(), "All Files (*);;PNG Files (*.png);;Jpg Files (*.jpg)")

        # Open The Image
        if fname:
            self.pixmap = QPixmap(fname[0])
            # Add Pic to label
            self.label.setPixmap(self.pixmap)

# Initialize The App
app = QApplication(sys.argv)
UIWindow = Window()
app.exec_()

0

In [1]:
from PyQt5 import QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QMenuBar, QMenu, QAction, QWidget,QFileDialog, QFrame
from PyQt5.QtGui import QIcon,QPixmap
from PyQt5 import QtCore,QtGui
import sys
import os

In [4]:
class PhotoViewer(QtWidgets.QGraphicsView):
    photoClicked = QtCore.pyqtSignal(QtCore.QPoint)

    def __init__(self, parent):
        super(PhotoViewer, self).__init__(parent)
        self._zoom = 0
        self._empty = True
        self._scene = QtWidgets.QGraphicsScene(self)
        self._photo = QtWidgets.QGraphicsPixmapItem()
        self._scene.addItem(self._photo)
        self.setScene(self._scene)
        self.setTransformationAnchor(QtWidgets.QGraphicsView.AnchorUnderMouse)
        self.setResizeAnchor(QtWidgets.QGraphicsView.AnchorUnderMouse)
        self.setVerticalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.setHorizontalScrollBarPolicy(QtCore.Qt.ScrollBarAlwaysOff)
        self.setBackgroundBrush(QtGui.QBrush(QtGui.QColor(30, 30, 30)))
        self.setFrameShape(QtWidgets.QFrame.NoFrame)
        self.setGeometry(2,parent.size().height(),694,520)
        
        
#     def resizeEvent(self, event=None):
#         self.fitInView(0, 0, 100, 100, Qt.KeepAspectRatio)
#         super().resizeEvent(event)

    def hasPhoto(self):
        return not self._empty

    def fitInView(self, scale=True):
        rect = QtCore.QRectF(self._photo.pixmap().rect())
        if not rect.isNull():
            self.setSceneRect(rect)
            if self.hasPhoto():
                unity = self.transform().mapRect(QtCore.QRectF(0, 0, 1, 1))
                self.scale(1 / unity.width(), 1 / unity.height())
                viewrect = self.viewport().rect()
                scenerect = self.transform().mapRect(rect)
                factor = min(viewrect.width() / scenerect.width(),
                             viewrect.height() / scenerect.height())
                self.scale(factor, factor)
            self._zoom = 0

    def setPhoto(self, pixmap=None):
        self._zoom = 0
        if pixmap and not pixmap.isNull():
            self._empty = False
            self.setDragMode(QtWidgets.QGraphicsView.ScrollHandDrag)
            self._photo.setPixmap(pixmap)
        else:
            self._empty = True
            self.setDragMode(QtWidgets.QGraphicsView.NoDrag)
            self._photo.setPixmap(QtGui.QPixmap())
        self.fitInView()

    def wheelEvent(self, event):
        if self.hasPhoto():
            if event.angleDelta().y() > 0:
                factor = 1.25
                self._zoom += 1
            else:
                factor = 0.8
                self._zoom -= 1
            if self._zoom > 0:
                self.scale(factor, factor)
            elif self._zoom == 0:
                self.fitInView()
            else:
                self._zoom = 0

    def toggleDragMode(self):
        if self.dragMode() == QtWidgets.QGraphicsView.ScrollHandDrag:
            self.setDragMode(QtWidgets.QGraphicsView.NoDrag)
        elif not self._photo.pixmap().isNull():
            self.setDragMode(QtWidgets.QGraphicsView.ScrollHandDrag)

    def mousePressEvent(self, event):
        if self._photo.isUnderMouse():
            self.photoClicked.emit(self.mapToScene(event.pos()).toPoint())
        super(PhotoViewer, self).mousePressEvent(event)


In [5]:
class Window(QMainWindow):
    def __init__(self, parent=None):
        """Initializer."""
        #super().__init__(parent)
        super(Window, self).__init__()
        self.setWindowTitle("Unraveling The Cytoskeleton")
        #self.setGeometry(200,200,300,300) 
        #self.centralWidget = QLabel("Hello, World")
        
        self.label = QLabel(self)
        #pixmap = QPixmap('./liver.bmp')
        #self.label.setPixmap(pixmap)
        #self.resize(pixmap.width(), pixmap.height())
        #self.centralWidget.setAlignment(Qt.AlignHCenter | Qt.AlignVCenter)
        #self.setCentralWidget(self.centralWidget)
        
        self._createActions()
        self._createMenuBar()
        self.viewer = PhotoViewer(self)
        self.viewer.photoClicked.connect(self.photoClicked)
        
    def _createActions(self):
        # Creating action using the first constructor
        self.newAction = QAction(self)
        self.newAction.setText("&New")
        self.openAction = QAction("&Open...", self)
        self.openAction.triggered.connect(self.openCall)
        self.saveAction = QAction("&Save", self)
        self.exitAction = QAction("&Exit", self)
        self.copyAction = QAction("&Copy", self)
        self.pasteAction = QAction("&Paste", self)
        self.cutAction = QAction("C&ut", self)
        self.helpContentAction = QAction("&Help Content", self)
        self.aboutAction = QAction("&About", self)
    
    def _createMenuBar(self):
        menuBar = self.menuBar()
        
        # FILE MENU
        fileMenu = QMenu("&File", self)
        menuBar.addMenu(fileMenu)
        fileMenu.addAction(self.newAction)
        fileMenu.addAction(self.openAction)
        fileMenu.addAction(self.saveAction)
        fileMenu.addAction(self.exitAction)
        
        # EDIT MENU
        editMenu = menuBar.addMenu("&Edit")
        editMenu.addAction(self.copyAction)
        editMenu.addAction(self.pasteAction)
        editMenu.addAction(self.cutAction)
        
        # HELP MENU
        helpMenu = menuBar.addMenu("&Help")
        helpMenu.addAction(self.helpContentAction)
        helpMenu.addAction(self.aboutAction)
        
    def photoClicked(self, pos):
        if self.viewer.dragMode()  == QtWidgets.QGraphicsView.NoDrag:
            self.editPixInfo.setText('%d, %d' % (pos.x(), pos.y()))
        
    def openCall(self):
        fname = QFileDialog.getOpenFileName(self, "Open File", os.getcwd() + "\\Datasets\\Set 1-a-tubulin_Sofia\\Deconvoluted", "All Files (*);;PNG Files (*.png);;JPG Files (*.jpg);;TIF Files (*.tif)")

        # Open The Image
        if fname:
            self.viewer.setPhoto(QPixmap(fname[0]))
#             self.pixmap.scaled(self.label.size())
#             self.label.setPixmap(self.pixmap)
            
#             self.label.adjustSize()
#             self.label.move(self.width()-self.label.height(), self.height() - self.label.width())
#             #self.label.setGeometry(self.width()-self.label.height(), self.height() - self.label.width(), 681, 551)
#             self.label.setFrameStyle(QLabel.StyledPanel)
#             self.label.setScaledContents(True)
#             self.label.setLineWidth(2)
#             #self.setCentralWidget(label)
#             #pixmap = QPixmap('./default.png')
#             #self.label.setPixmap(pixmap)
            
    

            
#########################################################
def main():
    app = QApplication(sys.argv)
    win = Window()
    win.showMaximized()
    sys.exit(app.exec_())
    
main()

SystemExit: 0

In [14]:
%tb

SystemExit: 0

In [12]:
# if __name__ == "__main__":
#     app = QApplication(sys.argv)
#     win = Window()
#     win.show()
#     sys.exit(app.exec_())



SystemExit: 0

In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
import os




class Window(QtWidgets.QWidget):
    def __init__(self):
        super(Window, self).__init__()
        self.viewer = PhotoViewer(self)
        # 'Load image' button
        self.btnLoad = QtWidgets.QToolButton(self)
        self.btnLoad.setText('Load image')
        self.btnLoad.clicked.connect(self.loadImage)
        # Button to change from drag/pan to getting pixel info
        self.btnPixInfo = QtWidgets.QToolButton(self)
        self.btnPixInfo.setText('Enter pixel info mode')
        self.btnPixInfo.clicked.connect(self.pixInfo)
        self.editPixInfo = QtWidgets.QLineEdit(self)
        self.editPixInfo.setReadOnly(True)
        self.viewer.photoClicked.connect(self.photoClicked)
        # Arrange layout
        VBlayout = QtWidgets.QVBoxLayout(self)
        VBlayout.addWidget(self.viewer)
        HBlayout = QtWidgets.QHBoxLayout()
        HBlayout.setAlignment(QtCore.Qt.AlignLeft)
        HBlayout.addWidget(self.btnLoad)
        HBlayout.addWidget(self.btnPixInfo)
        HBlayout.addWidget(self.editPixInfo)
        VBlayout.addLayout(HBlayout)

    def loadImage(self):
        self.viewer.setPhoto(QtGui.QPixmap(os.getcwd() + "\\Datasets\\Set 1-a-tubulin_Sofia\\Deconvoluted\\MAX_9_6IF_CHO_L_w3z_decon_ch02_PS.tif"))

    def pixInfo(self):
        self.viewer.toggleDragMode()

    def photoClicked(self, pos):
        if self.viewer.dragMode()  == QtWidgets.QGraphicsView.NoDrag:
            self.editPixInfo.setText('%d, %d' % (pos.x(), pos.y()))


if __name__ == '__main__':
    import sys
    app = QtWidgets.QApplication(sys.argv)
    window = Window()
    window.setGeometry(500, 300, 800, 600)
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\diogo\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3532: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
